In [ ]:
import app
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score

from sklearn.compose import ColumnTransformer
from tensorflow.keras import layers, optimizers

# Do extensions code below

# import dataset as pandas DataFrame
admissions = pd.read_csv('admissions_data.csv')

# inspecting the data
print(admissions.columns)
print(admissions.head())
print(admissions.describe())

# splitting data into feature parameters and labels
features = admissions.iloc[:, 1:-1]
labels = admissions.iloc[:, -1]

# splitting data into training and test sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=5)


# scale features

# initialize scaling object
ct = ColumnTransformer([('scale', StandardScaler(), features.columns)])

# fit ColumnTransformer to the training data and transform it
features_train_scaled = ct.fit_transform(features_train)
# transform test data using the trained ColumnTransformer
features_test_scaled = ct.transform(features_test)


# design deep learning model
def design_model(features, learning_rate):
    # initialize Sequential model
    model = Sequential()
    # create the input layer
    input = layers.InputLayer(input_shape=(features.shape[1],))
    # add the input layer to the model
    model.add(input)
    # add 3 hidden layers and 3 dropout layers
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(0.1))
    # add an output layer to the model
    model.add(layers.Dense(1))
    # initialize an Adam optimizer
    opt = optimizers.Adam(learning_rate=learning_rate)
    # compile model
    model.compile(loss='mse', metrics=['mae'], optimizer=opt)
    return model

learning_rate, epochs, batch_size= 0.001, 80, 1

model = design_model(features_train_scaled, learning_rate)
print(model.summary())

# initialize EarlyStopping that monitors the validation loss
stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# fit the model
history = model.fit(features_train_scaled, labels_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[stop])
# evaluate the model on the test data
val_mse, val_mae = model.evaluate(features_test_scaled, labels_test, verbose=0)


# evaluating R-squared score
y_predicted = model.predict(features_test_scaled)
print('R square= ', r2_score(labels_test,y_predicted))

# print validation mean squared error and mean absolute error
print('MSE = ', val_mse)
print('MAE =', val_mae)



# if you decide to do the Matplotlib extension, you must save your plot in the directory by uncommenting the line of code below

# loss vs epochs
fig, axs = plt.subplots(1, 2, gridspec_kw={'hspace': 1, 'wspace': 0.5}) 
(ax1, ax2) = axs
ax1.plot(history.history['loss'], label='train')
ax1.plot(history.history['val_loss'], label='validation')
ax1.set_title('lrate=' + str(learning_rate))
ax1.legend(loc="upper right")
ax1.set_xlabel("# of epochs")
ax1.set_ylabel("loss (mse)")

# mae vs epochs
ax2.plot(history.history['mae'], label='train')
ax2.plot(history.history['val_mae'], label='validation')
ax2.set_title('learning rate=' + str(learning_rate))
ax2.legend(loc="upper right")
ax2.set_xlabel("number of epochs")
ax2.set_ylabel("MAE")

fig.savefig('static/images/my_plots.png')